# Conifer parameter data from TRY

In [1]:
import pandas as pd
import os
#import myfuncs
from datetime import datetime
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 400)
from matplotlib import pyplot as plt

#define constants
m2_per_cm2 = 1e-4
g_biomass_per_g_C = 2
m2_per_mm2 = 1e-6
mg_per_g = 1e3
g_per_kg = 1000
mm2_per_cm2 = 100
g_per_mg = 1e-3
write_csv = True

In [23]:
path_to_trait_obs = "/home/adamhb/gdrive/postdoc/parameters/param_data/trait_observations"
write_to_csv = False

In [24]:
try_data = os.path.join(path_to_trait_obs,"conifer/TRY/conifer_traits_TRY.txt")

### Functions

In [25]:
def convert_to_fates_units(trait_name,input_units,input_value):

    
    if ("SLA" in trait_name) | ("Specific leaf area" in trait_name) and input_units == "mm2 mg-1":  
        output_value = input_value * m2_per_mm2 * mg_per_g * g_biomass_per_g_C
        return output_value

    elif "Leaf nitrogen" in trait_name and input_units == "mg/g":
        output_value = input_value * g_per_mg * g_biomass_per_g_C
        return output_value

    elif "Stem specific density" in trait_name and input_units == "g/cm3":
        output_value = input_value
        return output_value
    
    # Anaker conversions
    elif trait_name == "log(SLA cm2 g-1)":
        output_value = np.exp(input_value) * m2_per_cm2 * g_biomass_per_g_C
        return output_value
        
    elif trait_name == 'log(wood density g ml-1)':
        output_value = np.exp(input_value)
        return output_value
        
    elif trait_name == 'log(N %)':
        output_value = np.exp(input_value) * 1e-2 * g_biomass_per_g_C
        return output_value
    
    #Jepson leaf N
    elif trait_name == "Leaf Nmass":
    
        # muliplying by 1e-2 is to reverse the "per cent"  
        output_value = input_value * 1e-2 * g_biomass_per_g_C
        return output_value
    
    elif "Wood density" in trait_name:
        output_value = input_value
        return output_value
    
    elif trait_name == "Leaf area to sapwood area" and input_units == "mm2 mm-2":
        output_value = input_value * 1e-4
        return output_value
    
    elif "Vcmax" in trait_name and input_units == "micro mol m-2 s-1":
        output_value = input_value
        return output_value
    
    elif "hydraulic vulnerability" in trait_name and input_units == "Mpa":
        output_value = input_value
        return output_value
    
    elif "Leaf lifespan" in trait_name and input_units == "month":
        output_value = input_value / 12
        return output_value
    
    else:
        #print("No unit conversion known for:", trait_name)
        return None
    
def convert_to_fates_param_name(input_trait_name):
    if ("SLA" in input_trait_name) | ('Specific leaf area' in input_trait_name):
        return "fates_leaf_slatop"
    
    elif ("wood density" in input_trait_name) |  ("Wood density" in input_trait_name):
        return "fates_wood_density"
    
    elif ("N %" in input_trait_name) | ("Leaf Nmass" in input_trait_name):
        return "fates_stoich_nitr"
    
    elif "Leaf nitrogen (N) content per leaf dry mass" in input_trait_name:
        return "fates_stoich_nitr"
    
    elif "Leaf area to sapwood area" in input_trait_name:
        return "fates_allom_la_per_sa_int"
    
    elif "Vcmax" in input_trait_name:
        return "fates_leaf_vcmax25top"
    
    elif "hydraulic vulnerability" in input_trait_name:
        return "fates_nonhydro_smpsc"
    
    elif "Leaf lifespan" in input_trait_name:
        return "fates_turnover_leaf"
    
    
    else:
        return None
    
def get_sources(df,variable):
    return df.loc[df.fates_name == variable].source.unique()

def get_range_data(df,subset_name):

    summary_stats = df.groupby('fates_name')['fates_value'].agg(
        min_value='min',
        max_value='max',
        mean='mean',
        median='median',
        sample_size='size'
    )

    summary_stats['subset_name'] = subset_name
    
    return summary_stats

import numpy as np

def calculate_percentiles(data, percentiles):
    """
    Calculate the specified percentiles of a dataset.

    :param data: list or ndarray, the dataset
    :param percentiles: list of floats, the percentiles to compute, e.g., [25, 50, 75]
    :return: dictionary of percentiles and their corresponding values
    """
    results = {}
    for p in percentiles:
        value = np.percentile(data, p)
        results[p] = value
    return results

### Load, clean, and view oak trait data

In [26]:
# Load and join QUCH and QUKE data
try_df_raw = pd.read_csv(try_data, sep="\t", encoding='latin1')[['ObservationID','SpeciesName','AccSpeciesName','TraitID','TraitName','OrigValueStr','OrigUnitStr','StdValue','UnitName','ErrorRisk']]

try_df_raw = try_df_raw[['AccSpeciesName','TraitName','OrigValueStr','OrigUnitStr','StdValue','UnitName']]

print("Available trait observations")
print(try_df_raw.TraitName.value_counts())
#print("\n")
#print(try_df_raw.info())

Available trait observations
Plant height vegetative                                                                                    433
Leaf area per leaf dry mass (specific leaf area, SLA or 1/LMA): undefined if petiole is in- or excluded    259
Xylem hydraulic vulnerability, xylem cavitation vulnerability, embolism vulnerability, (P20, P50, P80)     116
Xylem hydraulic vulnerability curve (P20, P50, P80)                                                         98
Leaf area per leaf dry mass (specific leaf area, SLA or 1/LMA): petiole excluded                            90
Stem specific density (SSD, stem dry mass per stem fresh volume) or wood density                            74
Bark thickness                                                                                              68
Leaf nitrogen (N) content per leaf area                                                                     51
Leaf carbon (C) content per leaf area                                              

In [6]:
try_df_raw = pd.DataFrame.drop_duplicates(try_df_raw)

### Filter to relevant traits

In [7]:
# Filter to get traits of interest
fates_relevant_traits = ['Stem specific density (SSD, stem dry mass per stem fresh volume) or wood density',
                         'Leaf nitrogen (N) content per leaf dry mass',
                         'Leaf nitrogen (N) content per leaf area',
                         #'Bark thickness',
                         'Leaf area per leaf dry mass (specific leaf area, SLA or 1/LMA): undefined if petiole is in- or excluded',
                         'Leaf area per leaf dry mass (specific leaf area, SLA or 1/LMA): petiole excluded',
                         'Leaf lifespan (longevity)',
                         'Photosynthesis carboxylation capacity (Vcmax) per leaf area (Farquhar model)',
                         'Xylem hydraulic vulnerability, xylem cavitation vulnerability, embolism vulnerability, (P20, P50, P80)']

try_df = try_df_raw[try_df_raw.TraitName.isin(fates_relevant_traits)]
try_df.TraitName.value_counts()

Leaf area per leaf dry mass (specific leaf area, SLA or 1/LMA): undefined if petiole is in- or excluded    256
Xylem hydraulic vulnerability, xylem cavitation vulnerability, embolism vulnerability, (P20, P50, P80)     100
Leaf area per leaf dry mass (specific leaf area, SLA or 1/LMA): petiole excluded                            89
Stem specific density (SSD, stem dry mass per stem fresh volume) or wood density                            69
Leaf nitrogen (N) content per leaf area                                                                     31
Photosynthesis carboxylation capacity (Vcmax) per leaf area (Farquhar model)                                 2
Leaf lifespan (longevity)                                                                                    2
Name: TraitName, dtype: int64

### Get estimate of leaf nitrogen from leaf nitrogen per area

In [8]:
g_n_per_gC = try_df.loc[try_df['TraitName'].str.contains('nitrogen')]["StdValue"] * 0.010
#g_n_per_m2_leaf

calculate_percentiles(g_n_per_gC,[85,90,95,98])

{85: 0.046087210065167,
 90: 0.0465211635940981,
 95: 0.0498040957295726,
 98: 0.05235146694004941}

### Apply unit conversions and fates parameter names

In [9]:
# Unit convertion to fates units
fates_values = try_df.apply(lambda x: convert_to_fates_units(x['TraitName'], x['UnitName'], x["StdValue"]), axis=1)
try_df['fates_value'] = fates_values

# Convert variable names to fates names
fates_names = try_df.apply(lambda x: convert_to_fates_param_name(x["TraitName"]),axis = 1)
try_df["fates_name"] = fates_names


/tmp/ipykernel_18495/4023397515.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  try_df['fates_value'] = fates_values
/tmp/ipykernel_18495/4023397515.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  try_df["fates_name"] = fates_names


In [10]:
try_df.TraitName.unique()

array(['Leaf area per leaf dry mass (specific leaf area, SLA or 1/LMA): petiole excluded',
       'Leaf area per leaf dry mass (specific leaf area, SLA or 1/LMA): undefined if petiole is in- or excluded',
       'Stem specific density (SSD, stem dry mass per stem fresh volume) or wood density',
       'Leaf nitrogen (N) content per leaf area',
       'Photosynthesis carboxylation capacity (Vcmax) per leaf area (Farquhar model)',
       'Xylem hydraulic vulnerability, xylem cavitation vulnerability, embolism vulnerability, (P20, P50, P80)',
       'Leaf lifespan (longevity)'], dtype=object)

In [14]:
try_df.loc[try_df['TraitName' == 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549 entries, 9 to 24214
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AccSpeciesName  549 non-null    object 
 1   TraitName       549 non-null    object 
 2   OrigValueStr    525 non-null    object 
 3   OrigUnitStr     519 non-null    object 
 4   StdValue        525 non-null    float64
 5   UnitName        525 non-null    object 
 6   fates_value     494 non-null    float64
 7   fates_name      518 non-null    object 
dtypes: float64(2), object(6)
memory usage: 38.6+ KB


In [15]:
PIPO_df

,AccSpeciesName,TraitName,OrigValueStr,OrigUnitStr,StdValue,UnitName,fates_value,fates_name
9,Pinus ponderosa,Leaf area per leaf dry mass (specific leaf are...,165.4,cm2/g,16.540000,mm2 mg-1,0.033080,fates_leaf_slatop
15,Pinus ponderosa,Leaf area per leaf dry mass (specific leaf are...,96.6,cm2/g,9.660000,mm2 mg-1,0.019320,fates_leaf_slatop
26,Pinus ponderosa,Leaf area per leaf dry mass (specific leaf are...,16.5,mm2/mg,16.500000,mm2 mg-1,0.033000,fates_leaf_slatop
40,Pinus ponderosa,"Stem specific density (SSD, stem dry mass per ...",0.42,g/cm^3,0.420000,g/cm3,0.420000,fates_wood_density
141,Pinus ponderosa,"Stem specific density (SSD, stem dry mass per ...",350,kg/m3,0.350000,g/cm3,0.350000,fates_wood_density
174,Pinus ponderosa,Leaf area per leaf dry mass (specific leaf are...,275,g/m2,3.636364,mm2 mg-1,0.007273,fates_leaf_slatop
179,Pinus ponderosa,Leaf area per leaf dry mass (specific leaf are...,333.333333333333,g/m2,3.000000,mm2 mg-1,0.006000,fates_leaf_slatop
184,Pinus ponderosa,Leaf area per leaf dry mass (specific leaf are...,294.117647058824,g/m2,3.400000,mm2 mg-1,0.006800,fates_leaf_slatop
505,Pinus ponderosa,Leaf area per leaf dry mass (specific leaf are...,31.75,cm2/g,3.175000,mm2 mg-1,0.006350,fates_leaf_slatop
506,Pinus ponderosa,Leaf area per leaf dry mass (specific leaf are...,314.96062992126,g/m2,3.175000,mm2 mg-1,0.006350,fates_leaf_slatop


In [11]:
ABCO_df = try_df.loc[try_df.AccSpeciesName == "Abies concolor"]
CADE_df = try_df.loc[try_df.AccSpeciesName == "Calocedrus decurrens"]
PIPO_df = try_df.loc[try_df.AccSpeciesName == "Pinus ponderosa"]

param_range_conifer_df = pd.concat([get_range_data(ABCO_df,"ABCO"),get_range_data(CADE_df,"CADE"),get_range_data(PIPO_df,"PIPO")]).reset_index()
param_range_conifer_df['sources'] = "try"

### See percentiles of key parameters for PIPO

This is to make sure that the ranges we're testing based on Buotte et al., 2021 also line up well with the TRY data

In [13]:
param_range_conifer_df

,fates_name,min_value,max_value,mean,median,sample_size,subset_name,sources
0,fates_leaf_slatop,0.004412,0.016822,0.010273,0.009520,98,ABCO,try
1,fates_nonhydro_smpsc,-8.400000,-3.249000,-4.739429,-4.510000,14,ABCO,try
2,fates_turnover_leaf,7.700000,7.700000,7.700000,7.700000,1,ABCO,try
3,fates_wood_density,0.290600,0.486900,0.394754,0.397800,26,ABCO,try
4,fates_nonhydro_smpsc,-10.435596,-4.922196,-7.712140,-7.745384,5,CADE,try
5,fates_wood_density,0.260000,0.600000,0.392625,0.360000,8,CADE,try
6,fates_leaf_slatop,0.001880,0.033080,0.009243,0.007726,247,PIPO,try
7,fates_leaf_vcmax25top,48.720000,76.272000,62.496000,62.496000,2,PIPO,try
8,fates_nonhydro_smpsc,-5.448360,0.670000,-2.325816,-2.100000,81,PIPO,try
9,fates_turnover_leaf,3.000000,3.000000,3.000000,3.000000,1,PIPO,try


In [17]:
param_vars = ['fates_leaf_slatop', 'fates_leaf_vcmax25top', 'fates_nonhydro_smpsc']

for p in param_vars:
    print(p)
    print(PIPO_df.loc[PIPO_df['fates_name'].str.contains(p)].describe())

fates_leaf_slatop


ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [18]:
calculate_percentiles(PIPO_df.loc[PIPO_df['fates_name'] == "fates_leaf_slatop"]["fates_value"],[80,85,90,95,96,97,98])

{80: 0.01317679234368,
 85: 0.014330812676899998,
 90: 0.01519182041664,
 95: 0.016952663265931214,
 96: 0.017241379310353647,
 97: 0.01764018691585446,
 98: 0.019637883008360016}

### See percentiles of key parameters for CADE

This is to make sure that the ranges we're testing based on Buotte et al., 2021 also line up well with the TRY data

In [19]:
for p in param_vars:
    print(p)
    print(CADE_df.loc[CADE_df['fates_name'].str.contains(p)].describe())

fates_leaf_slatop
       StdValue  fates_value
count       0.0          0.0
mean        NaN          NaN
std         NaN          NaN
min         NaN          NaN
25%         NaN          NaN
50%         NaN          NaN
75%         NaN          NaN
max         NaN          NaN
fates_leaf_vcmax25top
       StdValue  fates_value
count       0.0          0.0
mean        NaN          NaN
std         NaN          NaN
min         NaN          NaN
25%         NaN          NaN
50%         NaN          NaN
75%         NaN          NaN
max         NaN          NaN
fates_nonhydro_smpsc
        StdValue  fates_value
count   4.000000     4.000000
mean   -7.712140    -7.712140
std     2.251164     2.251164
min   -10.435596   -10.435596
25%    -8.417937    -8.417937
50%    -7.745384    -7.745384
75%    -7.039587    -7.039587
max    -4.922196    -4.922196


### See percentiles of key parameters for ABCO

This is to make sure that the ranges we're testing based on Buotte et al., 2021 also line up well with the TRY data

In [20]:
for p in param_vars:
    print(p)
    print(ABCO_df.loc[ABCO_df['fates_name'].str.contains(p)].describe())

fates_leaf_slatop


ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [21]:
# Write CSV
if write_to_csv == True:

    now = datetime.now()
    dt_string = now.strftime("%m-%d-%Y-%H-%M-%S")
    file_path = "conifer_param_ranges_data_derived_" + dt_string + ".csv"
    param_range_conifer_df.to_csv(file_path, index=False)

In [ ]:
param_range_conifer_df